In [1]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np 
import time, math, random
from typing import Tuple

# import gym 
import gym

In [2]:
env = gym.make('CartPole-v0')

In [3]:
policy = lambda _,__,___, tip_velocity : int( tip_velocity > 0 )

In [4]:
n_bins = ( 6 , 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

def discretizer( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    """Convert continues state intro a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[angle, pole_velocity]])[0]))

In [5]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(6, 12, 2)

In [6]:
def policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])

In [7]:
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    """Temperal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [8]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [9]:
def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    """Decaying exploration rate"""
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [10]:
n_episodes = 10000 
for e in range(n_episodes):
    
    # Siscretize state into buckets
    current_state, done = discretizer(*env.reset()), False
    
    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _ = env.step(action)
        new_state = discretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()
    

Episode: 1 Time: 0.020000910758972167
Episode: 2 Time: 0.024999594688415526
Episode: 3 Time: 0.02330005168914795
Episode: 4 Time: 0.03340270519256592
Episode: 5 Time: 0.08670248985290527
Episode: 6 Time: 0.0967026948928833
Episode: 7 Time: 0.023299765586853028
Episode: 8 Time: 0.03499906063079834
Episode: 9 Time: 0.04006826877593994
Episode: 10 Time: 0.03840005397796631
Episode: 11 Time: 0.01650657653808594
Episode: 12 Time: 0.021701836585998537
Episode: 13 Time: 0.028309106826782227
Episode: 14 Time: 0.03512084484100342
Episode: 15 Time: 0.01829638481140137
Episode: 16 Time: 0.08178985118865967
Episode: 17 Time: 0.03659982681274414
Episode: 18 Time: 0.026700735092163086
Episode: 19 Time: 0.03319997787475586
Episode: 20 Time: 0.018199777603149413
Episode: 21 Time: 0.04159979820251465
Episode: 22 Time: 0.03169999122619629
Episode: 23 Time: 0.016499924659729003
Episode: 24 Time: 0.0199998140335083
Episode: 25 Time: 0.030002427101135255
Episode: 26 Time: 0.04159998893737793
Episode: 27 Ti

KeyboardInterrupt: 

In [ ]:
env.close()